# Multimodal House Price Valuation — Preprocessing & EDA

This notebook sets up the **data foundations** for a multimodal house price model using:
- **Tabular attributes** (King County housing data)
- **Satellite imagery** (Sentinel Hub tiles around each property)

The objectives here are:
- To make our **economic and causal assumptions explicit**.
- To perform **tabular + geospatial EDA** that challenges those assumptions.
- To prepare **clean, leakage-aware splits** for downstream modeling.

We treat this as a production-grade analytical system, not a Kaggle notebook; every modeling choice must earn its right to exist.



## Economic and Causal Assumptions

We model **transaction prices** as an equilibrium outcome of supply and demand, conditional on both **property attributes** and **neighbourhood quality**.

Tabular variables (bedrooms, bathrooms, square footage, grade, condition, waterfront, view, latitude/longitude, etc.) capture only part of this structure. They typically miss:

- **Greenery and open space**: tree canopy, parks, and visual openness vs. concrete.
- **Water proximity and views**: lakes, rivers, ocean, marinas beyond a coarse `waterfront` flag.
- **Road density and accessibility**: cul-de-sacs vs. arterial roads, distance to main corridors.
- **Urban texture and density**: detached houses vs. townhouses vs. apartment blocks.
- **Neighbourhood planning and layout**: block shapes, lot regularity, street grid vs. organic patterns.

Satellite imagery may act as a **proxy** for these latent neighbourhood factors. We are **not** claiming pixels cause higher prices; instead, we test whether visual context contains **predictive information** beyond the tabular features.

Key risks and caveats:
- **Spatial confounding**: high-demand areas may have both higher prices and better amenities; imagery cannot de-confound this.
- **Measurement error** in coordinates and imagery (clouds, acquisition time, seasonal effects).
- **Selection bias** if our imagery coverage differs systematically across locations.

Our evaluation will therefore focus on **out-of-sample predictive performance** and **stability across spatial splits**, not causal effects.


## Data Loading and Paths

We assume the project root contains the following files provided by you:

- `train.csv` – training data with the **target price**.
- `test.xlsx` – test data **without price** (features only, for blind prediction).
- `APIcredentials/` – text file with Sentinel Hub credentials (used via `.env` or environment variables).

To make the notebook robust, we:
- Look for data under `data/raw/` if you follow the recommended structure.
- Fall back to the project root if the files are there instead.

If your column names differ from the canonical King County dataset (e.g., `price`, `id`, `lat`, `long`), adjust the configuration cell below accordingly.


In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import geopandas as gpd
import contextily as ctx

plt.style.use("seaborn-v0_8")
sns.set_context("talk")

PROJECT_ROOT = Path("..").resolve()
RAW_DIR = PROJECT_ROOT / "data" / "raw"
PROCESSED_DIR = PROJECT_ROOT / "data" / "processed"
SATELLITE_DIR = PROJECT_ROOT / "data" / "satellite"
EMBEDDINGS_DIR = PROJECT_ROOT / "data" / "embeddings"
PROVIDED_DIR = PROJECT_ROOT / "Provided"

for d in [RAW_DIR, PROCESSED_DIR, SATELLITE_DIR, EMBEDDINGS_DIR]:
    d.mkdir(parents=True, exist_ok=True)

train_candidates = [
    RAW_DIR / "train.csv",
    PROJECT_ROOT / "train.csv",
    PROVIDED_DIR / "train.csv",
]

test_candidates = [
    RAW_DIR / "test.xlsx",
    PROJECT_ROOT / "test.xlsx",
    PROVIDED_DIR / "test.xlsx",
]

for p in train_candidates:
    if p.exists():
        TRAIN_PATH = p
        break
else:
    raise FileNotFoundError(
        f"Could not find train.csv in {train_candidates}. Please adjust paths."
    )

for p in test_candidates:
    if p.exists():
        TEST_PATH = p
        break
else:
    raise FileNotFoundError(
        f"Could not find test.xlsx in {test_candidates}. Please adjust paths."
    )

print("Using TRAIN_PATH =", TRAIN_PATH)
print("Using TEST_PATH =", TEST_PATH)

train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_excel(TEST_PATH)

print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)

train_df.head()


In [ ]:
# Configure key column names (adjust here if your schema differs)

# Attempt to infer common column names; override manually if needed.

def infer_first_present(candidates, columns, default=None):
    for c in candidates:
        if c in columns:
            return c
    return default

cols = train_df.columns

TARGET_COL = infer_first_present(["price", "SalePrice", "y"], cols)
ID_COL = infer_first_present(["id", "Id", "house_id"], cols)
LAT_COL = infer_first_present(["lat", "latitude"], cols)
LON_COL = infer_first_present(["long", "lon", "longitude"], cols)

if TARGET_COL is None:
    raise ValueError(
        "Could not infer target column (price). Set TARGET_COL manually in this cell."
    )
if ID_COL is None:
    raise ValueError(
        "Could not infer ID column. Set ID_COL manually in this cell."
    )
if LAT_COL is None or LON_COL is None:
    raise ValueError(
        "Could not infer latitude/longitude columns. Set LAT_COL and LON_COL manually."
    )

print("TARGET_COL =", TARGET_COL)
print("ID_COL =", ID_COL)
print("LAT_COL =", LAT_COL)
print("LON_COL =", LON_COL)

# Create a log-price for more Gaussian-like residuals
train_df["log_price"] = np.log1p(train_df[TARGET_COL])

train_df[[TARGET_COL, "log_price"]].describe().T


In [ ]:
# 1️⃣ Tabular EDA: price distributions and key drivers

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

sns.histplot(train_df[TARGET_COL], bins=50, ax=axes[0])
axes[0].set_title("Raw price distribution")
axes[0].set_xlabel(TARGET_COL)

sns.histplot(train_df["log_price"], bins=50, ax=axes[1])
axes[1].set_title("Log(price + 1) distribution")
axes[1].set_xlabel("log_price")

plt.tight_layout()
plt.show()

# Simple relationship plots for selected drivers (if present)
key_drivers = [
    "sqft_living",
    "sqft_lot",
    "sqft_living15",
    "sqft_lot15",
    "bedrooms",
    "bathrooms",
    "grade",
    "condition",
]

available_drivers = [c for c in key_drivers if c in train_df.columns]

print("Using key drivers:", available_drivers)

fig, axes = plt.subplots(len(available_drivers), 1, figsize=(10, 4 * len(available_drivers)))
if len(available_drivers) == 1:
    axes = [axes]

for ax, col in zip(axes, available_drivers):
    sns.scatterplot(
        data=train_df.sample(min(5000, len(train_df)), random_state=42),
        x=col,
        y="log_price",
        alpha=0.3,
        ax=ax,
    )
    ax.set_title(f"log_price vs {col}")

plt.tight_layout()
plt.show()


In [ ]:
# 1️⃣ Tabular EDA: correlation and multicollinearity

numeric_cols = train_df.select_dtypes(include=[np.number]).columns.tolist()

# Drop obvious non-features from correlation view
for col in [ID_COL]:
    if col in numeric_cols:
        numeric_cols.remove(col)

corr = train_df[numeric_cols].corr()

plt.figure(figsize=(12, 10))
mask = np.triu(np.ones_like(corr, dtype=bool))
sns.heatmap(corr, mask=mask, cmap="coolwarm", center=0, square=True, cbar_kws={"shrink": 0.5})
plt.title("Correlation matrix (numeric features)")
plt.show()

# Variance Inflation Factor (VIF) for a subset of core regressors
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif_features = [
    c
    for c in [
        "sqft_living",
        "sqft_lot",
        "sqft_living15",
        "sqft_lot15",
        "bedrooms",
        "bathrooms",
        "grade",
        "condition",
    ]
    if c in train_df.columns
]

if len(vif_features) > 1:
    X_vif = train_df[vif_features].dropna().astype(float)
    vif_data = []
    for i, col in enumerate(vif_features):
        vif_value = variance_inflation_factor(X_vif.values, i)
        vif_data.append({"feature": col, "VIF": vif_value})
    vif_df = pd.DataFrame(vif_data).sort_values("VIF", ascending=False)
    display(vif_df)
else:
    print("Not enough features for VIF calculation; adjust vif_features if needed.")


In [ ]:
# 1️⃣ Tabular EDA: monotonicity sanity checks

# We expect average log_price to increase (weakly) with size and quality.

features_to_check = [
    f for f in ["sqft_living", "sqft_lot", "grade", "bathrooms", "bedrooms"] if f in train_df.columns
]

def plot_monotonicity(feature, bins=10):
    tmp = train_df[[feature, "log_price"]].dropna().copy()
    try:
        # Use quantile bins to get roughly equal-sized groups
        tmp["bin"] = pd.qcut(tmp[feature], q=bins, duplicates="drop")
    except ValueError:
        # Fallback to simple bins for discrete features like grade
        tmp["bin"] = pd.cut(tmp[feature], bins=min(bins, tmp[feature].nunique()))
    gp = tmp.groupby("bin")["log_price"].mean()
    gp.plot(kind="bar", figsize=(8, 4))
    plt.title(f"Average log_price by binned {feature}")
    plt.ylabel("mean(log_price)")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.show()

for f in features_to_check:
    plot_monotonicity(f)


## Geospatial EDA: spatial structure and leakage risk

We now examine the **spatial structure** of prices using latitude/longitude:

- Are high prices spatially clustered (e.g., along waterfronts or affluent suburbs)?
- Do residuals from a simple tabular model show **spatial patterns**, indicating missing spatial features?
- How should we define **train/validation splits** to avoid training on near-identical neighbours of test points?

We use GeoPandas and a web-mercator projection to overlay price patterns on a basemap.


In [ ]:
# 2️⃣ Geospatial EDA: price maps and clustering intuition

# Build a GeoDataFrame

gdf = gpd.GeoDataFrame(
    train_df.copy(),
    geometry=gpd.points_from_xy(train_df[LON_COL], train_df[LAT_COL]),
    crs="EPSG:4326",
)

# Project to web-mercator for plotting with contextily

gdf_web = gdf.to_crs(epsg=3857)

# Subsample for faster plotting if dataset is large
max_points = 8000
if len(gdf_web) > max_points:
    gdf_web_sample = gdf_web.sample(max_points, random_state=42)
else:
    gdf_web_sample = gdf_web

fig, ax = plt.subplots(1, 1, figsize=(10, 10))

price_col = "log_price"

gdf_web_sample.plot(
    ax=ax,
    column=price_col,
    cmap="viridis",
    scheme="quantiles",
    k=10,
    markersize=5,
    alpha=0.7,
    legend=True,
)

# Use a widely available tile provider; some Stamen tiles are deprecated in newer contextily versions
ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)
ax.set_axis_off()
ax.set_title("Spatial distribution of log_price (sampled)")
plt.tight_layout()
plt.show()

# Simple spatial clustering proxy: bin lat/long into a grid to approximate neighbourhoods

grid_size_deg = 0.02  # ~2 km; adjust if your area is more compact or spread out

lat_bin = (train_df[LAT_COL] / grid_size_deg).round().astype(int)
lon_bin = (train_df[LON_COL] / grid_size_deg).round().astype(int)

train_df["spatial_bin"] = lat_bin.astype(str) + "_" + lon_bin.astype(str)

print("Number of spatial bins:", train_df["spatial_bin"].nunique())
train_df["spatial_bin"].value_counts().head()


## Visual EDA: satellite tiles by price quantile

Before training any image model, we want to **look at the data like a human appraiser**:

- For low-, mid-, and high-priced properties, what does the surrounding environment look like?
- Do expensive homes visually cluster near **water**, **greenery**, or low-density cul-de-sacs?
- Are there systematic artefacts (clouds, missing tiles) that could mislead a model?

We sample satellite tiles (if already fetched via `data_fetcher.py`) and group them by price quantiles to form initial **visual hypotheses**.


In [ ]:
from PIL import Image

meta_path = SATELLITE_DIR / "image_metadata.csv"

if not meta_path.exists():
    print(
        f"No satellite metadata found at {meta_path}. "
        "Run data_fetcher.py first to download Sentinel tiles."
    )
else:
    meta_df = pd.read_csv(meta_path)
    # Keep successful or cached images only
    meta_df = meta_df[meta_df["status"].isin(["ok", "cached"])]

    merged = train_df.merge(
        meta_df[["id", "image_path"]].rename(columns={"id": ID_COL}),
        on=ID_COL,
        how="left",
    )

    # Define price quantiles
    merged["price_quantile"] = pd.qcut(
        merged[TARGET_COL], q=[0.0, 0.2, 0.8, 1.0], labels=["low", "mid", "high"]
    )

    def show_samples_for_quantile(label, n=9):
        subset = merged[(merged["price_quantile"] == label) & merged["image_path"].notna()]
        if subset.empty:
            print(f"No images found for quantile '{label}'.")
            return
        sample = subset.sample(min(n, len(subset)), random_state=42)
        n_cols = 3
        n_rows = int(np.ceil(len(sample) / n_cols))
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(4 * n_cols, 4 * n_rows))
        axes = axes.flatten()
        for ax, (_, row) in zip(axes, sample.iterrows()):
            try:
                img = Image.open(row["image_path"]).convert("RGB")
                ax.imshow(img)
                ax.set_title(f"{label} price\nID={row[ID_COL]}")
                ax.axis("off")
            except Exception as e:  # noqa: BLE001
                ax.text(0.5, 0.5, f"Error: {e}", ha="center", va="center")
                ax.axis("off")
        for ax in axes[len(sample) :]:
            ax.axis("off")
        plt.suptitle(f"Satellite samples for {label}-priced properties")
        plt.tight_layout()
        plt.show()

    for q in ["low", "mid", "high"]:
        show_samples_for_quantile(q, n=9)


## Leakage-aware train/validation splits

A naïve random train/test split on individual rows can **leak spatial information**:

- Nearby properties (same street or block) share unobserved neighbourhood factors.
- If one home is in train and its neighbour is in test, the model may **memorise local price levels** rather than learn generalisable relationships.

To mitigate this, we:

- Use the `spatial_bin` grid defined above as a proxy for neighbourhood clusters.
- Apply a **group-wise split** so that all properties in a given spatial bin fall entirely in train or validation.
- Keep your original `test.xlsx` as a final blind set (features only); we do **not** leak its information into training.


In [ ]:
from sklearn.model_selection import GroupShuffleSplit

# Ensure spatial_bin exists
if "spatial_bin" not in train_df.columns:
    raise RuntimeError("spatial_bin not defined; run the geospatial EDA cell first.")

groups = train_df["spatial_bin"].astype(str)

gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, val_idx = next(gss.split(train_df, groups=groups))

train_split = train_df.iloc[train_idx].copy()
val_split = train_df.iloc[val_idx].copy()

print("Train split shape:", train_split.shape)
print("Validation split shape:", val_split.shape)

# Save processed splits for modeling notebook
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

train_split.to_parquet(PROCESSED_DIR / "train.parquet", index=False)
val_split.to_parquet(PROCESSED_DIR / "val.parquet", index=False)

print("Saved train/val splits to", PROCESSED_DIR)
